In [1]:
import pandas as pd
import pandas as pd
from collections import Counter
from tqdm import tqdm
from nltk.tokenize import TweetTokenizer
import nltk
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re
import gensim
import gensim.corpora as corpora
from pprint import pprint
from nltk.stem import WordNetLemmatizer


words = set(nltk.corpus.words.words())

In [2]:
df = pd.read_csv('tweet_all.csv',lineterminator="\n")

In [3]:
df.shape

(1810294, 10)

In [4]:
df.head(5)

,tweet_id,combined_text,time,favourites_count,followers_count,friends_count,statuses_count,rt_status,state,token_list
0,1223471140317909004,Veteran journo canned after making ‘racist’ jo...,02-01-2020,28128,4478,4826,42564,0,Tennessee,"['veteran', 'journo', 'racist', 'joke', 'https..."
1,1223471149142560773,UCSF expert working on quicker diagnostic test...,02-01-2020,2158,1126,1882,729,1,NaN,"['ucsf', 'expert', 'work', 'quicker', 'diagnos..."
2,1223471417578139649,RT @YahooFinance: Dr. Oz explains how to stay ...,02-01-2020,162241,264,292,168875,1,New York,"['@yahoofinance', 'dr', 'oz', 'explains', 'sta..."
3,1223471598755368960,Here's what did it. 👀👀👀👀👀👀\nCoronavirus? Bill ...,02-01-2020,4843,2613,3867,28471,1,Texas,"[""here's"", 'bill', 'gate', 'patent', 'run', 's..."
4,1223471814753443841,#BREAKING @Qantas will suspend flights to and ...,02-01-2020,553,1641,204,3788,0,Wisconsin,"['#breaking', '@qantas', 'suspend', 'flight', ..."


In [5]:
## date time type
df['time'] = pd.to_datetime(df['time'])
df['time'] = df['time'].astype('str')

In [6]:
# convert to list
import ast
df['token_list'] = df['token_list'].apply(lambda x: ast.literal_eval(x))

In [7]:
#df = df.sample(1000)

In [8]:
df['month'] = df['time'].str.slice(0,7)

In [9]:
df.groupby('month').count()

,tweet_id,combined_text,time,favourites_count,followers_count,friends_count,statuses_count,rt_status,state,token_list
month,,,,,,,,,,
2020-02,41262,41262,41262,41262,41262,41262,41262,41262,35002,41262
2020-03,202267,202267,202267,202267,202267,202267,202267,202267,180232,202267
2020-04,146554,146554,146554,146554,146554,146554,146554,146554,128957,146554
2020-05,115856,115856,115856,115856,115856,115856,115856,115856,102053,115856
2020-06,79311,79311,79311,79311,79311,79311,79311,79311,70866,79311
2020-07,106044,106044,106044,106044,106044,106044,106044,106044,94109,106044
2020-08,104811,104811,104811,104811,104811,104811,104811,104811,92964,104811
2020-09,90855,90855,90855,90855,90855,90855,90855,90855,80558,90855
2020-10,133935,133935,133935,133935,133935,133935,133935,133935,118409,133935


# process token_list

In [10]:
def process_token(list):
    result = []
    for each in list:
        if each in words:
            result.append(each)
    return result
        

In [11]:
df['filter_token'] = df['token_list'].apply(process_token)

In [12]:
df

,tweet_id,combined_text,time,favourites_count,followers_count,friends_count,statuses_count,rt_status,state,token_list,month,filter_token
0,1223471140317909004,Veteran journo canned after making ‘racist’ jo...,2020-02-01,28128,4478,4826,42564,0,Tennessee,"[veteran, journo, racist, joke, https://t.co/v...",2020-02,"[veteran, racist, joke]"
1,1223471149142560773,UCSF expert working on quicker diagnostic test...,2020-02-01,2158,1126,1882,729,1,NaN,"[ucsf, expert, work, quicker, diagnostic, test...",2020-02,"[expert, work, diagnostic, test, able, diagnos..."
2,1223471417578139649,RT @YahooFinance: Dr. Oz explains how to stay ...,2020-02-01,162241,264,292,168875,1,New York,"[@yahoofinance, dr, oz, explains, stay, health...",2020-02,"[stay, healthy, amid, outbreak]"
3,1223471598755368960,Here's what did it. 👀👀👀👀👀👀\nCoronavirus? Bill ...,2020-02-01,4843,2613,3867,28471,1,Texas,"[here's, bill, gate, patent, run, simulation, ...",2020-02,"[bill, gate, patent, run, simulation, month, p..."
4,1223471814753443841,#BREAKING @Qantas will suspend flights to and ...,2020-02-01,553,1641,204,3788,0,Wisconsin,"[#breaking, @qantas, suspend, flight, mainland...",2020-02,"[suspend, flight, outbreak]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1810289,1421680845602050051,- The locking up of children in cold tight qua...,2021-08-01,217764,1761,2775,55936,1,Indiana,"[lock, child, cold, tight, quarter, normalize,...",2021-08,"[lock, child, cold, tight, quarter, normalize,..."
1810290,1421681168584478722,@FoxNews As covid rages the state. Good one De...,2021-08-01,2062,48,279,7915,0,Colorado,"[@foxnews, rage, state, good, deathsantis]",2021-08,"[rage, state, good]"
1810291,1421681267125460995,@CoreyBrickley GIVE ME MOAR COVID JUICE,2021-08-01,37459,2593,2783,12406,0,Pennsylvania,"[@coreybrickley, moar, juice]",2021-08,[juice]
1810292,1421681374919024640,Tuberculosis:The Forgotten Pandemic\nhttps://t...,2021-08-01,368,146,413,7888,1,New Jersey,"[tuberculosis, forgotten, https://t.co/rm4sotz...",2021-08,"[tuberculosis, forgotten, tuberculosis, forgot..."


In [27]:
import re
# replaces spaces in keywords with connector matcher.
def replace_connector(keyword):
  connector = "[ -_]?"
  return connector.join(keyword.split())

#Filter tweets when keywords mention both covid and vaccine(not specify vaccine name)
covid_keywords = ["covid 19", "covid 2019", "covid", "19 ncov", "2019 ncov", "ncov", "corona virus", "pandemic", "wuhan virus", "chinese virus", "china virus", "sars cov 2", "hcov 19", "ncov 19"]
covid_query = r"\b(" + "|".join(map(replace_connector, covid_keywords)) + r")\b"
covid_reg = re.compile(covid_query, re.IGNORECASE)

vaccine_keywords = ["vaccine", "vaccination","vaxx"]
vaccine_query = r"\b(" + "|".join(map(replace_connector, vaccine_keywords)) + r")\b"
vaccine_reg = re.compile(vaccine_query, re.IGNORECASE)

#Filter tweets when keywords mention specified vaccine name related to covid
vac_name_keywords = [i.lower() for i in ["Pfizer", "BNT162b2", "AstraZeneca", "AZD1222", "Moderna", "mRNA-1273", "Janssen", "Ad26.COV2.S"]]
vac_name_query = r"\b(" + "|".join(map(replace_connector, vac_name_keywords)) + r")\b"
vac_name_reg = re.compile(vac_name_query, re.IGNORECASE)


def text_keyword_filter(text):
    if vac_name_reg.search(text):
        return True;
    return False
   

In [28]:
# mask = df[['location']].apply(lambda x : x.str.contains(patt)).any(axis=1)

# df_usa = df[mask]
mask = df['combined_text'].apply(lambda x : text_keyword_filter(x))
df_mask = df[mask]

In [29]:
df

,tweet_id,combined_text,time,favourites_count,followers_count,friends_count,statuses_count,rt_status,state,token_list,month,filter_token
0,1223471140317909004,Veteran journo canned after making ‘racist’ jo...,2020-02-01,28128,4478,4826,42564,0,Tennessee,"[veteran, journo, racist, joke, https://t.co/v...",2020-02,"[veteran, racist, joke]"
1,1223471149142560773,UCSF expert working on quicker diagnostic test...,2020-02-01,2158,1126,1882,729,1,NaN,"[ucsf, expert, work, quicker, diagnostic, test...",2020-02,"[expert, work, diagnostic, test, able, diagnos..."
2,1223471417578139649,RT @YahooFinance: Dr. Oz explains how to stay ...,2020-02-01,162241,264,292,168875,1,New York,"[@yahoofinance, dr, oz, explains, stay, health...",2020-02,"[stay, healthy, amid, outbreak]"
3,1223471598755368960,Here's what did it. 👀👀👀👀👀👀\nCoronavirus? Bill ...,2020-02-01,4843,2613,3867,28471,1,Texas,"[here's, bill, gate, patent, run, simulation, ...",2020-02,"[bill, gate, patent, run, simulation, month, p..."
4,1223471814753443841,#BREAKING @Qantas will suspend flights to and ...,2020-02-01,553,1641,204,3788,0,Wisconsin,"[#breaking, @qantas, suspend, flight, mainland...",2020-02,"[suspend, flight, outbreak]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1810289,1421680845602050051,- The locking up of children in cold tight qua...,2021-08-01,217764,1761,2775,55936,1,Indiana,"[lock, child, cold, tight, quarter, normalize,...",2021-08,"[lock, child, cold, tight, quarter, normalize,..."
1810290,1421681168584478722,@FoxNews As covid rages the state. Good one De...,2021-08-01,2062,48,279,7915,0,Colorado,"[@foxnews, rage, state, good, deathsantis]",2021-08,"[rage, state, good]"
1810291,1421681267125460995,@CoreyBrickley GIVE ME MOAR COVID JUICE,2021-08-01,37459,2593,2783,12406,0,Pennsylvania,"[@coreybrickley, moar, juice]",2021-08,[juice]
1810292,1421681374919024640,Tuberculosis:The Forgotten Pandemic\nhttps://t...,2021-08-01,368,146,413,7888,1,New Jersey,"[tuberculosis, forgotten, https://t.co/rm4sotz...",2021-08,"[tuberculosis, forgotten, tuberculosis, forgot..."


In [30]:
df_mask

,tweet_id,combined_text,time,favourites_count,followers_count,friends_count,statuses_count,rt_status,state,token_list,month,filter_token
8169,1225870363566989312,JUST IN: The head of the NIH says no glitches ...,2020-02-07,30720,328,2373,55388,1,Missouri,"[head, nih, glitch, far, development, moderna'...",2020-02,"[head, far, development, potential, vaccine, h..."
8286,1225906155681128449,#Feb7th ‘No Glitches’ in Early Effort for Coro...,2020-02-07,973,220,428,99275,0,New York,"[#feb7th, glitch, early, effort, vaccine, nih,...",2020-02,"[early, effort, vaccine, stock, fly]"
11881,1227269478976671746,#Biotech company $mrna Moderna successfully co...,2020-02-11,42,3807,2936,703,0,California,"[#biotech, company, mrna, moderna, successfull...",2020-02,"[company, successfully, collaborate, develop, ..."
15325,1228379903323590656,AstraZeneca stock slips on mixed earnings repo...,2020-02-14,1263,730,1742,187439,0,Indiana,"[astrazeneca, stock, slip, mixed, earnings, re...",2020-02,"[stock, slip, mixed, earnings, report, uncerta..."
19316,1229837785500200961,NEWS: As part of the rapid global response to ...,2020-02-18,791,2298,1027,5482,1,New York,"[news, rapid, global, response, 2019, novel, h...",2020-02,"[news, rapid, global, response, novel, use, ex..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1809289,1421546853540241411,And a whole bunch of preventable deaths due to...,2021-07-31,37029,664,1502,17289,1,Massachusetts,"[bunch, preventable, death, #deltavariant, fol...",2021-07,"[bunch, preventable, death, follow, week, dos,..."
1809449,1421567751643140097,RT @atomic_79: Canada buys 65M Pfizer booster ...,2021-07-31,20903,6254,6707,20286,1,Nevada,"[@atomic_79, canada, buy, 65m, pfizer, booster...",2021-07,"[canada, buy, booster, shot, protection, variant]"
1809573,1421584542733946881,Former Pfizer Employee Confirms Poison in COVI...,2021-07-31,1428,632,1828,99389,0,Minnesota,"[pfizer, employee, confirms, poison, vaccine, ...",2021-07,"[employee, poison, vaccine]"
1809653,1421593486223159297,@lauvysht If you have Pfizer as an option cons...,2021-07-31,5642,164,159,23003,0,Florida,"[@lauvysht, pfizer, option, consider, it's, he...",2021-07,"[option, consider, hesitation, march]"


## Sentiment

In [44]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [85]:
s1 = 'today sux!'

In [86]:
s2 = 'todau SUX!'

In [87]:
sid.polarity_scores(s1)

{'neg': 0.736, 'neu': 0.264, 'pos': 0.0, 'compound': -0.4199}

In [88]:
sid.polarity_scores(s2)

{'neg': 0.779, 'neu': 0.221, 'pos': 0.0, 'compound': -0.5461}

In [32]:
df_mask['scores'] = df_mask['combined_text'].apply(lambda text: sid.polarity_scores(text))

C:\Users\74871\AppData\Local\Temp/ipykernel_16700/2930331554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mask['scores'] = df_mask['combined_text'].apply(lambda text: sid.polarity_scores(text))


In [33]:
df_mask['compound']  = df_mask['scores'].apply(lambda score_dict: score_dict['compound'])

C:\Users\74871\AppData\Local\Temp/ipykernel_16700/3542552229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mask['compound']  = df_mask['scores'].apply(lambda score_dict: score_dict['compound'])


In [34]:
df_mask['comp_score'] = df_mask['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

C:\Users\74871\AppData\Local\Temp/ipykernel_16700/1701598875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mask['comp_score'] = df_mask['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')


In [35]:
df_mask.sample(1)

,tweet_id,combined_text,time,favourites_count,followers_count,friends_count,statuses_count,rt_status,state,token_list,month,filter_token,scores,compound,comp_score
1455964,1363898507300642820,RT @juliabhaber: Does anyone know of anyone ge...,2021-02-22,175649,5071,5534,288078,1,New York,"[@juliabhaber, know, arm, pfizer, vaccine, mod...",2021-02,"[know, arm, vaccine]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,pos


In [39]:
import numpy as np
df1 = pd.pivot_table(df_mask, values='compound', index=['month'], aggfunc=np.mean)

In [40]:
df1.sample(1)

,compound
month,
2021-04,0.112223


In [41]:
df1 = df1.reset_index()

In [42]:
df1

,month,compound
0,2020-02,0.238039
1,2020-03,0.229700
2,2020-04,0.206995
3,2020-05,0.170837
4,2020-06,0.102975
5,2020-07,0.192758
6,2020-08,0.178707
7,2020-09,-0.025593
8,2020-10,0.012389
9,2020-11,0.255261


In [43]:
df1.to_csv('sentiment_vacc.csv',index=False)

In [78]:
df.groupby(['month','state','comp_score']).comp_score.count().to_frame()

comp_score
month   state      comp_score            
2020-02 Alabama    neg                242
                   pos                283
        Alaska     neg                 58
                   pos                 80
        Arizona    neg                352
...                                   ...
2021-08 Washington neg                 12
                   pos                 12
        Wisconsin  neg                  2
                   pos                  4
        Wyoming    pos                  2

[1908 rows x 1 columns]

In [72]:
df.columns

Index(['tweet_id', 'combined_text', 'time', 'favourites_count',
       'followers_count', 'friends_count', 'statuses_count', 'rt_status',
       'state', 'token_list', 'month', 'filter_token', 'scores', 'compound',
       'comp_score'],
      dtype='object')

In [73]:
df1 = df[['time','state','month','compound','comp_score']]

In [74]:
df1.to_csv('sentiment.csv')